In [1]:
import mysql.connector
import pandas as pd
import statistics as st
from datetime import datetime as dt
from datetime import timedelta
db_connection = mysql.connector.connect(user='root', password='Gregory2',host='localhost',
                                     port='3306', database='pbl3210 db2')

In [2]:
stockidtab=pd.read_sql_query("SELECT * FROM STOCK_ID",db_connection)
stockpricetab=pd.read_sql_query("SELECT * FROM STOCKPRICE",db_connection)
stockearningstab=pd.read_sql_query("SELECT * FROM EARNINGS",db_connection)


In [3]:
zeelist=[]
for i in stockearningstab["z_score"]:
    try:
        zeelist.append(float(i))
    except TypeError:
        zeelist.append(i)
stockearningstab["z_score"]=zeelist
   

In [4]:
def stockvaluetable(Ticker):
    for i in stockidtab.index:
        if stockidtab.at[i,"ticker"]==Ticker:
            id_value=stockidtab.at[i,"stock_id"]
    df=stockpricetab[stockpricetab["stock_id"]==id_value]
    df.index=df["Date"]
    return df

In [5]:
def stocksurprisetable(Ticker):
    for i in stockidtab.index:
        if stockidtab.at[i,"ticker"]==Ticker:
            id_value=stockidtab.at[i,"stock_id"]
    df2=stockearningstab[stockearningstab["stock_id"]==id_value]
    return df2


In [48]:
Ticker=input("Please input the ticker you are looking for\n")

Please input the ticker you are looking for
NKE


In [49]:
values=stockvaluetable(Ticker)
surprises=stocksurprisetable(Ticker)
print(values)

            stock_id        Date     Price
Date                                      
1990-01-02       345  1990-01-02    0.8789
1990-01-03       345  1990-01-03    0.8594
1990-01-04       345  1990-01-04    0.8418
1990-01-05       345  1990-01-05    0.8672
1990-01-08       345  1990-01-08    0.8633
...              ...         ...       ...
2020-02-12       345  2020-02-12  103.0000
2020-02-13       345  2020-02-13  103.3700
2020-02-14       345  2020-02-14  103.5400
2020-02-18       345  2020-02-18  102.0000
2020-02-19       345  2020-02-19  102.4600

[7592 rows x 3 columns]


In [50]:
z_score_standardised=[]
z_score_values=[]
for i in surprises.index:
    z_score_values.append(surprises.at[i,"z_score"])
mean=st.mean(z_score_values)   
sd=st.stdev(z_score_values)
for i in z_score_values:
    z_score_standardised.append((i-mean)/sd)
z_score_standardised  
surprises["z_score_standardised"]=z_score_standardised

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [51]:
dirlist=[]
for i in surprises.index:
    if surprises.loc[i,"z_score"]==0:
        dirlist.append("No Surprise")
    elif surprises.loc[i,"z_score"]>0:
        dirlist.append("Positive")
    elif surprises.loc[i,"z_score"]<0:
        dirlist.append("Negative")
    else:
        dirlist.append(None)
        

In [52]:
surprises["Direction"]=dirlist

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [54]:
maglist=[]
for i in z_score_values:
    if i==0:
        maglist.append("No Surprise")
    elif abs(i)>0:
        if abs(i)<=1:
            maglist.append("Small")
        elif abs(i)>1:
            if abs(i)<=2:
                maglist.append("Medium")
            else:
                maglist.append("Large")
    else:
        maglist.append(None)
      
surprises["Magnitude"]=maglist  
surprises
for i in surprises:
    print(i)

stock_id
Quarter
Date
Median
Actual
StdDev
z_score
z_score_standardised
Direction
Magnitude


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [56]:
#surprises and values, Date and Price
direc=input("Positive or Negative?")
mag=input("Small, Medium or Large?")
for i in surprises.index:
        if surprises.loc[i]["Direction"]==direc and surprises.loc[i]["Magnitude"]==mag:
            try:
                print("On",surprises.loc[i]["Date"],"\n The one day price change was",values.at[(surprises.loc[i]["Date"]+timedelta(days=1)),"Price"]-values.at[surprises.loc[i]["Date"],"Price"],"\n The 20 day price change was",values.at[(surprises.loc[i]["Date"]+timedelta(days=20)), "Price"]-values.at[surprises.loc[i]["Date"], "Price"],"\n And the 100 day price change was",values.at[(surprises.loc[i]["Date"]+timedelta(days=100)), "Price"]-values.at[surprises.loc[i]["Date"], "Price"],"\n")
            except KeyError:
                try:
                    print("On",surprises.loc[i]["Date"],"\n The one day price change was",values.at[(surprises.loc[i]["Date"]+timedelta(days=1)),"Price"]-values.at[surprises.loc[i]["Date"],"Price"],"\n And the 20 day price change was",values.at[(surprises.loc[i]["Date"]+timedelta(days=50)), "Price"]-values.at[surprises.loc[i]["Date"], "Price"],"\n")
                except KeyError:
                    try:
                        print("On",surprises.loc[i]["Date"],"\n The one day price change was",values.at[(surprises.loc[i]["Date"]+timedelta(days=1)),"Price"]-values.at[surprises.loc[i]["Date"],"Price"],"\n")
                    except KeyError:
                        continue
                          

Positive or Negative?Positive
Small, Medium or Large?Small
On 1999-01-14 
 The one day price change was -0.10939999999999994 
 And the 20 day price change was 1.5077999999999996 

On 2001-03-20 
 The one day price change was 0.2737000000000007 
 The 20 day price change was 0.18620000000000037 
 And the 100 day price change was 0.4962 

On 2001-06-28 
 The one day price change was 0.001300000000000523 
 And the 20 day price change was 0.8438000000000008 

On 2002-03-21 
 The one day price change was -0.46250000000000036 
 And the 20 day price change was -1.2875000000000005 

On 2002-09-18 
 The one day price change was 0.11999999999999922 
 The 20 day price change was -0.3800000000000008 
 And the 100 day price change was 0.05999999999999961 

On 2002-12-19 
 The one day price change was 0.44620000000000015 
 And the 20 day price change was 0.1086999999999998 

On 2005-03-17 
 The one day price change was -0.17879999999999896 
 And the 20 day price change was -1.1312999999999995 

On 20